In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Reading corpus the text file
with open("/content/sample_data/IndiaUS.txt", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()

In [ ]:
mytext

'Following a lavish state visit by Indian Prime Minister Narendra Modi to Washington, US President Joe Biden has called his country\'s partnership with India among the "most consequential in the world". The BBC\'s Vikas Pandey and Soutik Biswas explore the factors that contribute to the visit\'s potential in strengthening the ties between the two nations.\n\nThe US\'s relationship with India - the world\'s most populous country - is "stronger, closer and more dynamic than any time in history", Mr Biden said at the completion of a pomp-filled state visit by Mr Modi to the White House.\n\nThe remark may not be an exaggeration. "This summit suggests that the relationship has been transformed. It underscores just how broad and deep it has become in a relatively short time," says Michael Kugelman of The Wilson Center, an American think-tank.\n\nA key reason is that Washington is keen to draw India closer so that it can act as a counterbalance to China\'s growing influence in the Indo-Pacifi

In [ ]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [ ]:
print(total_words)
mytokenizer.word_index

In [ ]:
my_input_sequences = []
for line in mytext.split('\n'):
    #print(line)
    token_list = mytokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        #print(my_n_gram_sequence)
        my_input_sequences.append(my_n_gram_sequence)
        print(input_sequences)

In [ ]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
input_sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  99,   4, 177], dtype=int32)

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
X[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 99,  4],
      dtype=int32)

In [ ]:
y[1]

177

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
y[0]

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 100)           59900     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 599)               90449     
                                                                 
Total params: 300949 (1.15 MB)
Trainable params: 300949 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

In [ ]:
input_text = "Joe biden"
predict_next_words= 6

for _ in range(predict_next_words):
    token_list = mytokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in mytokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

print(input_text)

In [ ]:
input_text = "Joe biden"
predict_next_words= 6
for _ in range(predict_next_words):
    token_list = mytokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)

In [ ]:
predicted.argmax()

5